In [2]:
PATH = '/Users/siddhantthakur/pizza req/'

import json
import matplotlib.pyplot as plt
import pandas as pd
import nltk

In [69]:
with open(PATH + 'pizza_request_dataset/pizza_request_dataset.json', 'r') as f:
    data = json.load(f)

In [70]:
len(data)

5671

In [71]:
sorted(data[0].keys())

['giver_username_if_known',
 'in_test_set',
 'number_of_downvotes_of_request_at_retrieval',
 'number_of_upvotes_of_request_at_retrieval',
 'post_was_edited',
 'request_id',
 'request_number_of_comments_at_retrieval',
 'request_text',
 'request_text_edit_aware',
 'request_title',
 'requester_account_age_in_days_at_request',
 'requester_account_age_in_days_at_retrieval',
 'requester_days_since_first_post_on_raop_at_request',
 'requester_days_since_first_post_on_raop_at_retrieval',
 'requester_number_of_comments_at_request',
 'requester_number_of_comments_at_retrieval',
 'requester_number_of_comments_in_raop_at_request',
 'requester_number_of_comments_in_raop_at_retrieval',
 'requester_number_of_posts_at_request',
 'requester_number_of_posts_at_retrieval',
 'requester_number_of_posts_on_raop_at_request',
 'requester_number_of_posts_on_raop_at_retrieval',
 'requester_number_of_subreddits_at_request',
 'requester_received_pizza',
 'requester_subreddits_at_request',
 'requester_upvotes_minus

In [72]:
success = [d['requester_received_pizza'] for d in data]
print("Average Success Rate",str(sum(success)/len(success)*100))

#imbalanced dataset

Average Success Rate 24.634103332745546


In [73]:
data[5004]

{'giver_username_if_known': 'N/A',
 'in_test_set': True,
 'number_of_downvotes_of_request_at_retrieval': 4,
 'number_of_upvotes_of_request_at_retrieval': 33,
 'post_was_edited': False,
 'request_id': 't3_pcmhh',
 'request_number_of_comments_at_retrieval': 1,
 'request_text': "So, yes. We both work retail to make ends meet, and normally this works out for us, but given that it's early in the year and retail money sucks, we were budgeted to just barely scrape by. Now, with this development, I had to use my last $20 for her Metro fare so she can get to work. That was gonna go towards PB sammiches. Whoops.\n\nThis all seems like a sob story more worthy of the pantry or assistance, but it's no life or death situation. Reddit-flavored pizza would brighten the week for us till we get paid on Friday, but we'll still be around otherwise.",
 'request_text_edit_aware': "So, yes. We both work retail to make ends meet, and normally this works out for us, but given that it's early in the year and re

In [78]:
for d in data:
    d['upvotes_minus_downvotes'] = d['number_of_upvotes_of_request_at_retrieval'] - d['number_of_downvotes_of_request_at_retrieval']